## Passo 1: limpa os ISBNs e agrupa os repetidos

Retira tudo o que não é digito e "apanha" apenas o primeiro (13 digitos)

In [ ]:
import pandas as pd
import re

# Read the CSV file
df = pd.read_csv('Lista de novos items-reportresults.csv', delimiter=';')

# Filter rows where itemcallnumber is not null
# df = df[df['itemcallnumber'].notna()]

# Function to sanitize ISBN
def sanitize_isbn(isbn):
    # Remove any character that is not a digit
    sanitized = re.sub(r'\D', '', isbn)
    # Only keep the first 13 digits (standard length of ISBN)
    return sanitized[:13]

# Apply the function to the ISBN column
df['ISBN'] = df['ISBN'].apply(sanitize_isbn)

# Convert the ISBN column to a list
isbn_list = df['ISBN'].tolist()

isbn_list = list(set(isbn_list))

print(isbn_list)


## Vai buscar os dados ao Google Books via API

... se não encontrar no Google Books, vai tentar obter esses dados da Wook:
https://www.wook.pt/pesquisa?keyword=<isbn>

In [30]:
import requests

def get_book_info(isbn):
    # Send a GET request to the Google Books API
    response = requests.get(f'https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}')

    # Check that the request was successful
    if response.status_code == 200:
        # Parse the response as JSON
        data = response.json()

        # Get the first book from the results
        try:
            book = data['items'][0]['volumeInfo']
            # Get the required information
            title = book.get('title')
            authors = book.get('authors')
            publisher = book.get('publisher')
            publishedDate = book.get('publishedDate')
            language = book.get('language')
            summary = book.get('description')
            thumbnail = book.get('imageLinks', {}).get('thumbnail')
            previewLink = book.get('previewLink')

            return title, authors, publisher, publishedDate, language, summary, thumbnail, previewLink

        except (IndexError, KeyError):
            print('______________________________________________')
            print(f'Não encontrei dados para o ISBN: {isbn}')
            print(f'... será que a wook o tem? https://www.wook.pt/pesquisa?keyword={isbn}')
            
    else:
        print(f'Request failed with status code {response.status_code}')
        return None

for isbn in isbn_list:  
        # Exemplo:
        try:
            book_info = get_book_info(isbn)
            if book_info is not None:
                print('______________________________________________')
                print(f'Dados Google Books para o ISBN: {isbn}')
                print('----------------------------------------------')
                title, authors, publisher, publishedDate, language, summary, thumbnail, previewLink = book_info
                print(f'* Título: {title}')
                if authors is not None:
                    print(f'* Autor(es): {", ".join(authors)}')
                print(f'* Editora: {publisher}')
                print(f'* Data de Publicação: {publishedDate}')
                print(f'* Lingua: {language}')
                print(f'* Resumo: {summary}')
                print(f'* Capa: {thumbnail}')
                print(f'* Pré-Visualização: {previewLink}')
                print(f'* Registo no Catálogo UA: https://opac.ua.pt/cgi-bin/koha/opac-search.pl?idx=nb&q={isbn}')
        except (IndexError, KeyError):
            print("")

______________________________________________
Dados Google Books para o ISBN: 9783030572860
----------------------------------------------
* Título: Industrial Organization
* Autor(es): Pak-Sing Choi, Eric Dunaway, Felix Muñoz-Garcia
* Editora: Springer
* Data de Publicação: 2022-01-21
* Lingua: en
* Resumo: This textbook presents 122 exercises on industrial organization with detailed answer keys. While most textbooks on industrial organization focus on theory and empirical findings, this textbook offers practical examples and exercises helping predict firm behaviour in different industries. The book emphasizes the game-theoretic tools used in each type of exercise, so students can systematically apply them to other markets, forms of competition, or information environments where firms, consumers, and regulating agencies interact. The book begins with examples that analyse different models of firm behavior and interaction; starting with monopoly and moving through the Cournot model of